# Install Requirements

In [1]:
!pip install -r requirements.txt

# Get Data

In [2]:
# 2016
dataset_name = "ISIC_2016"
train_zip_url = "https://isic-challenge-data.s3.amazonaws.com/2016/ISBI2016_ISIC_Part1_Training_Data.zip"
test_zip_url = "https://isic-challenge-data.s3.amazonaws.com/2016/ISBI2016_ISIC_Part1_Test_Data.zip"
mask_train_zip_url = "https://isic-challenge-data.s3.amazonaws.com/2016/ISBI2016_ISIC_Part1_Training_GroundTruth.zip"
mask_test_zip_url = "https://isic-challenge-data.s3.amazonaws.com/2016/ISBI2016_ISIC_Part1_Test_GroundTruth.zip"
train_path = "data/train"
test_path = "data/test"
mask_train_path = "data/mask_train"
mask_test_path = "data/mask_test"

In [3]:
!python data/preprocessing.py \
--dataset_name $dataset_name \
--train_zip_url $train_zip_url \
--test_zip_url $test_zip_url \
--mask_train_zip_url $mask_train_zip_url \
--mask_test_zip_url $mask_test_zip_url \
--train_path $train_path \
--test_path $test_path \
--mask_train_path $mask_train_path \
--mask_test_path $mask_test_path

/home/ai/miniconda3/envs/tf-gpu/lib/python3.9/site-packages/deep_utils/vision/face_detection/ultralight/__init__.py:3: UserWarning: 
No module named 'deep_utils.vision.face_detection.ultralight.torch'. If you don't use UltralightTorchFaceDetector ignore this message.
  UltralightTorchFaceDetector = import_module(
/home/ai/miniconda3/envs/tf-gpu/lib/python3.9/site-packages/deep_utils/vision/face_detection/ultralight/__init__.py:6: UserWarning: 
No module named 'deep_utils.vision.face_detection.ultralight.tf'. If you don't use UltralightTFFaceDetector ignore this message.
  UltralightTFFaceDetector = import_module(
[INFO] zip file: ISBI2016_ISIC_Part1_Training_Data doesn't exist! Unzipping ISBI2016_ISIC_Part1_Training_Data.zip
[INFO] Successfully extracted ISBI2016_ISIC_Part1_Training_Data.zip to .!
[INFO] zip file: ISBI2016_ISIC_Part1_Training_GroundTruth doesn't exist! Unzipping ISBI2016_ISIC_Part1_Training_GroundTruth.zip
[INFO] Successfully extracted ISBI2016_ISIC_Part1_Training_Grou

# Training Models
### Parameters

In [1]:
import shutil
import os
from utils.group_metrics import get_conf_mean_std

In [3]:
# basic
dataset_name = "ISIC_2016"
model_names = [
    'r2unet_cbam',
    'r2unet',
    'unet_res50',
    'unet_conv_deconv',
    'unet_attention_gate', # This model is not deterministic
    'unet_cbam',
    'unet_cbam_gate',
    'unet_pyramid_cbam_gate',
    "mcg_unet",
    'double_unet',
    ]
               
save_path = "saved_models/"
train_identifier = "usual"
multi_train = 1 # Run model for several times to estmiate std and mean metrics!
epochs = 1
batch_size = 32
img_size = (256, 256)
img_channel = 3
loss = "dice_loss"

# learning rate
lr = 1e-3
min_lr = 1e-5

# callbacks
reduce_lr_patience = 15
reduce_lr_factor = 0.1
early_stopping_p = 30

seeds = 1234
verbose = 1

# Augmentation
hair_aug_p = 0
hair_rmv_p = 0

mosaic_p = 0
cutmix_p = 0
cutmix_beta = 0

# Constant
hue_shift_limit = 1
sat_shift_limit = 0
contrast_limit = 0.1
brightness_limit = 0.1
hue_p = 0.5
contrast_p = 0.5
brightness_p = 0.5
random_rotate_p = 0.5
p_horizontal_flip = 0.5
p_vertical_flip = 0.5
p_center_crop = 0.5

metrics= (
    "dice_loss", 
    "dice_score",
    "focal_tversky_loss",
    "iou",
    "jaccard_loss",
    "loss",
    "val_dice_loss",
    "val_dice_score",
    "val_focal_tversky_loss",
    "val_iou",
    "val_jaccard_loss",
    "val_loss"
         )

In [5]:
# Ensure reproducibility
# !PYTHONHASHSEED=0
# !TF_DETERMINISTIC_OPS=0
# !TF_CUDNN_DETERMINISTIC=0

In [6]:
import shutil
import os
from utils.group_metrics import get_conf_mean_std

for model in model_names:
    model_path = os.path.join(save_path, dataset_name, model)
    for n in range(multi_train):
        seed = seeds + n
        saved_path_name = f"{train_identifier}_{n}"
        iter_model = os.path.join(model_path, saved_path_name)
        print(f"[INFO] Removing old model checkpoints: {iter_model}!!")
        shutil.rmtree(iter_model) if os.path.isdir(iter_model) else print(f'[INFO] {iter_model} does not exist!')
        !python train.py \
        --model $model --dataset_name $dataset_name \
        --save_path $save_path \
        --save_path_name $saved_path_name \
        --epochs $epochs \
        --lr $lr \
        --min_lr $min_lr \
        --reduce_lr_patience $reduce_lr_patience \
        --reduce_lr_factor $reduce_lr_factor \
        --early_stopping_p $early_stopping_p \
        --hair_aug_p $hair_aug_p \
        --hair_rmv_p $hair_rmv_p \
        --random_rotate_p $random_rotate_p \
        --p_horizontal_flip $p_horizontal_flip \
        --p_vertical_flip $p_vertical_flip \
        --p_center_crop $p_center_crop \
        --mosaic_p $mosaic_p \
        --cutmix_p $cutmix_p \
        --cutmix_beta $cutmix_beta \
        --seed $seed
    
        print(f"\n-----------------------------train {n} is done! for model: {model} ----------------------------------\n")
    try:
        csv_addresses = [
 os.path.join(save_path, dataset_name, model, f"{train_identifier}_{n}", "exp_1" , "csv_logger_train.csv") for n in range(multi_train)   
]
        metrics = get_mean_std(csv_addresses, 
                       arguments=(),
                       operators=(min, max, min, max, min, min, min, max, min, max, min, min),
                       save_path=os.path.join(save_path, dataset_name, model, "mean_std_metrics.csv")
                      )
        for name, metric in metrics.items():
            print(f"[INFO] Model: {model} == > {name}: {metric}")
    except Exception as e:
        print(f"[ERROR] {e} raised for {model_path}")

[INFO] Removing old model checkpoints: saved_models/ISIC_2016/r2unet_cbam/usual_0!!
[INFO] Successfully created logger for SKIN-CANCER-saved_models/ISIC_2016/r2unet_cbam/usual_0/exp_1
[INFO] Saving params!
2022-02-28 20:27:34,088 - SKIN-CANCER-saved_models/ISIC_2016/r2unet_cbam/usual_0/exp_1 - INFO - [INFO] Params are successfully saved in saved_models/ISIC_2016/r2unet_cbam/usual_0/exp_1/params.txt!
2022-02-28 20:27:34,088 - SKIN-CANCER-saved_models/ISIC_2016/r2unet_cbam/usual_0/exp_1 - INFO - Chosen Model: r2unet_cbam
2022-02-28 20:27:34,088 - SKIN-CANCER-saved_models/ISIC_2016/r2unet_cbam/usual_0/exp_1 - INFO -  Arguments: Namespace(optimizer='adam', batch_size=8, img_channel=3, transfer_learning_epochs=5, finetuning_epochs=10, epochs=1, lr=0.001, min_lr=1e-05, reduce_lr_patience=15, reduce_lr_factor=0.1, early_stopping_p=30, mlflow_source='./mlruns', cutmix_p=0.0, cutmix_beta=0.0, usual_aug_with_cutmix=False, hair_aug_p=0.0, hair_rmv_p=0.0, random_rotate_p=0.5, p_horizontal_flip=0.5

90/90 [==============================] - ETA: 0s - loss: 0.5765 - dice_score: 0.4235 - iou: 0.2721 - dice_loss: 0.5765 - jaccard_loss: 0.7200 - focal_tversky_loss: 0.5538
Epoch 00001: val_loss improved from inf to 0.58363, saving model to saved_models/ISIC_2016/r2unet/usual_0/exp_1/model
90/90 [==============================] - 36s 356ms/step - loss: 0.5765 - dice_score: 0.4235 - iou: 0.2721 - dice_loss: 0.5765 - jaccard_loss: 0.7200 - focal_tversky_loss: 0.5538 - val_loss: 0.5836 - val_dice_score: 0.4164 - val_iou: 0.2690 - val_dice_loss: 0.5836 - val_jaccard_loss: 0.7270 - val_focal_tversky_loss: 0.5587 - lr: 0.0010
2022-02-28 20:29:22,164 - SKIN-CANCER-saved_models/ISIC_2016/r2unet/usual_0/exp_1 - INFO - Training is done model is saved in saved_models/ISIC_2016/r2unet/usual_0/exp_1
2022-02-28 20:29:23,448 - SKIN-CANCER-saved_models/ISIC_2016/r2unet/usual_0/exp_1 - INFO - Successfully saved figures to saved_models/ISIC_2016/r2unet/usual_0/exp_1/images/metrics!
Evaluation
48/48 [=====

2022-02-28 20:30:37,937 - SKIN-CANCER-saved_models/ISIC_2016/unet_conv_deconv/usual_0/exp_1 - INFO - Data Loader is successfully loaded!
2022-02-28 20:30:39,531 - SKIN-CANCER-saved_models/ISIC_2016/unet_conv_deconv/usual_0/exp_1 - INFO - Successfully visualized input samples!
2022-02-28 20:30:41,351 - SKIN-CANCER-saved_models/ISIC_2016/unet_conv_deconv/usual_0/exp_1 - INFO - Successfully visualized input samples!
2022-02-28 20:30:42,283 - SKIN-CANCER-saved_models/ISIC_2016/unet_conv_deconv/usual_0/exp_1 - INFO - Successfully visualized input samples!
2022-02-28 20:30:42,657 - SKIN-CANCER-saved_models/ISIC_2016/unet_conv_deconv/usual_0/exp_1 - INFO - model: unet_conv_deconv is successfully loaded!
[INFO] Successfully created following callbacks [model_checkpoint, reduce_lr, early_stop, csv_logger, tensorboard]
2022-02-28 20:30:42,726 - SKIN-CANCER-saved_models/ISIC_2016/unet_conv_deconv/usual_0/exp_1 - INFO - loss: dice_loss is successfully created!
2022-02-28 20:30:42,729 - SKIN-CANCER

2022-02-28 20:32:04,805 - SKIN-CANCER-saved_models/ISIC_2016/unet_attention_gate/usual_0/exp_1 - INFO - Successfully Saved figures in saved_models/ISIC_2016/unet_attention_gate/usual_0/exp_1/images/test!

-----------------------------train 0 is done! for model: unet_attention_gate ----------------------------------

[ERROR] An error raised for saved_models/ISIC_2016/unet_attention_gate
[INFO] Removing old model checkpoints: saved_models/ISIC_2016/unet_cbam/usual_0!!
[INFO] saved_models/ISIC_2016/unet_cbam/usual_0 does not exist!
[INFO] Successfully created logger for SKIN-CANCER-saved_models/ISIC_2016/unet_cbam/usual_0/exp_1
[INFO] Saving params!
2022-02-28 20:32:07,219 - SKIN-CANCER-saved_models/ISIC_2016/unet_cbam/usual_0/exp_1 - INFO - [INFO] Params are successfully saved in saved_models/ISIC_2016/unet_cbam/usual_0/exp_1/params.txt!
2022-02-28 20:32:07,220 - SKIN-CANCER-saved_models/ISIC_2016/unet_cbam/usual_0/exp_1 - INFO - Chosen Model: unet_cbam
2022-02-28 20:32:07,220 - SKIN-CAN

90/90 [==============================] - ETA: 0s - loss: 0.5722 - dice_score: 0.4278 - iou: 0.2758 - dice_loss: 0.5722 - jaccard_loss: 0.5674 - focal_tversky_loss: 0.6115
Epoch 00001: val_loss improved from inf to 0.65738, saving model to saved_models/ISIC_2016/unet_cbam_gate/usual_0/exp_1/model
90/90 [==============================] - 26s 251ms/step - loss: 0.5722 - dice_score: 0.4278 - iou: 0.2758 - dice_loss: 0.5722 - jaccard_loss: 0.5674 - focal_tversky_loss: 0.6115 - val_loss: 0.6574 - val_dice_score: 0.3426 - val_iou: 0.2086 - val_dice_loss: 0.6574 - val_jaccard_loss: 0.6579 - val_focal_tversky_loss: 0.6900 - lr: 0.0010
2022-02-28 20:33:28,326 - SKIN-CANCER-saved_models/ISIC_2016/unet_cbam_gate/usual_0/exp_1 - INFO - Training is done model is saved in saved_models/ISIC_2016/unet_cbam_gate/usual_0/exp_1
2022-02-28 20:33:29,586 - SKIN-CANCER-saved_models/ISIC_2016/unet_cbam_gate/usual_0/exp_1 - INFO - Successfully saved figures to saved_models/ISIC_2016/unet_cbam_gate/usual_0/exp_1

2022-02-28 20:34:40,294 - SKIN-CANCER-saved_models/ISIC_2016/mcg_unet/usual_0/exp_1 - INFO - Data Loader is successfully loaded!
2022-02-28 20:34:42,015 - SKIN-CANCER-saved_models/ISIC_2016/mcg_unet/usual_0/exp_1 - INFO - Successfully visualized input samples!
2022-02-28 20:34:43,642 - SKIN-CANCER-saved_models/ISIC_2016/mcg_unet/usual_0/exp_1 - INFO - Successfully visualized input samples!
2022-02-28 20:34:44,588 - SKIN-CANCER-saved_models/ISIC_2016/mcg_unet/usual_0/exp_1 - INFO - Successfully visualized input samples!
2022-02-28 20:34:45,071 - SKIN-CANCER-saved_models/ISIC_2016/mcg_unet/usual_0/exp_1 - INFO - model: mcg_unet is successfully loaded!
[INFO] Successfully created following callbacks [model_checkpoint, reduce_lr, early_stop, csv_logger, tensorboard]
2022-02-28 20:34:45,140 - SKIN-CANCER-saved_models/ISIC_2016/mcg_unet/usual_0/exp_1 - INFO - loss: dice_loss is successfully created!
2022-02-28 20:34:45,145 - SKIN-CANCER-saved_models/ISIC_2016/mcg_unet/usual_0/exp_1 - INFO - 

2022-02-28 20:36:19,600 - SKIN-CANCER-saved_models/ISIC_2016/double_unet/usual_0/exp_1 - INFO - predicting for plots!
1/1 [==============================] - 1s 1s/step - loss: 0.6378 - dice_score: 0.3622 - iou: 0.2211 - dice_loss: 0.6378 - jaccard_loss: 0.7169 - focal_tversky_loss: 0.6841
2022-02-28 20:36:28,806 - SKIN-CANCER-saved_models/ISIC_2016/double_unet/usual_0/exp_1 - INFO - predicting for plots!
2022-02-28 20:36:29,305 - SKIN-CANCER-saved_models/ISIC_2016/double_unet/usual_0/exp_1 - INFO - Successfully Saved figures in saved_models/ISIC_2016/double_unet/usual_0/exp_1/images/test!

-----------------------------train 0 is done! for model: double_unet ----------------------------------

[ERROR] An error raised for saved_models/ISIC_2016/double_unet


## With cutmix and Mosaic

In [4]:
train_identifier = "usual_cutmix_mosaic"

# Augmentation
hair_aug_p = 0
hair_rmv_p = 0

mosaic_p = 0.5
cutmix_p = 0.5
cutmix_beta = 1


In [ ]:
import shutil
import os
from utils.group_metrics import get_conf_mean_std

for model in model_names:
    model_path = os.path.join(save_path, dataset_name, model)
    for n in range(multi_train):
        seed = seeds + n
        saved_path_name = f"{train_identifier}_{n}"
        iter_model = os.path.join(model_path, saved_path_name)
        print(f"[INFO] Removing old model checkpoints: {iter_model}!!")
        shutil.rmtree(iter_model) if os.path.isdir(iter_model) else print(f'[INFO] {iter_model} does not exist!')
        !python train.py \
        --model $model --dataset_name $dataset_name \
        --save_path $save_path \
        --save_path_name $saved_path_name \
        --epochs $epochs \
        --lr $lr \
        --min_lr $min_lr \
        --reduce_lr_patience $reduce_lr_patience \
        --reduce_lr_factor $reduce_lr_factor \
        --early_stopping_p $early_stopping_p \
        --hair_aug_p $hair_aug_p \
        --hair_rmv_p $hair_rmv_p \
        --random_rotate_p $random_rotate_p \
        --p_horizontal_flip $p_horizontal_flip \
        --p_vertical_flip $p_vertical_flip \
        --p_center_crop $p_center_crop \
        --mosaic_p $mosaic_p \
        --cutmix_p $cutmix_p \
        --cutmix_beta $cutmix_beta \
        --seed $seed
    
        print(f"\n-----------------------------train {n} is done! for model: {model} ----------------------------------\n")
    try:
        csv_addresses = [
 os.path.join(save_path, dataset_name, model, f"{train_identifier}_{n}", "exp_1" , "csv_logger_train.csv") for n in range(multi_train)   
]
        metrics = get_mean_std(csv_addresses, 
                       arguments=(),
                       operators=(min, max, min, max, min, min, min, max, min, max, min, min),
                       save_path=os.path.join(save_path, dataset_name, model, "mean_std_metrics.csv")
                      )
        for name, metric in metrics.items():
            print(f"[INFO] Model: {model} == > {name}: {metric}")
    except Exception as e:
        print(f"[ERROR] {e} raised for {model_path}")

[INFO] Removing old model checkpoints: saved_models/ISIC_2016/r2unet_cbam/usual_cutmix_mosaic_0!!
[INFO] Successfully created logger for SKIN-CANCER-saved_models/ISIC_2016/r2unet_cbam/usual_cutmix_mosaic_0/exp_1
[INFO] Saving params!
2022-02-28 20:41:13,662 - SKIN-CANCER-saved_models/ISIC_2016/r2unet_cbam/usual_cutmix_mosaic_0/exp_1 - INFO - [INFO] Params are successfully saved in saved_models/ISIC_2016/r2unet_cbam/usual_cutmix_mosaic_0/exp_1/params.txt!
2022-02-28 20:41:13,662 - SKIN-CANCER-saved_models/ISIC_2016/r2unet_cbam/usual_cutmix_mosaic_0/exp_1 - INFO - Chosen Model: r2unet_cbam
2022-02-28 20:41:13,662 - SKIN-CANCER-saved_models/ISIC_2016/r2unet_cbam/usual_cutmix_mosaic_0/exp_1 - INFO -  Arguments: Namespace(optimizer='adam', batch_size=8, img_channel=3, transfer_learning_epochs=5, finetuning_epochs=10, epochs=1, lr=0.001, min_lr=1e-05, reduce_lr_patience=15, reduce_lr_factor=0.1, early_stopping_p=30, mlflow_source='./mlruns', cutmix_p=0.5, cutmix_beta=1.0, usual_aug_with_cutm

90/90 [==============================] - ETA: 0s - loss: 0.5735 - dice_score: 0.4265 - iou: 0.2747 - dice_loss: 0.5735 - jaccard_loss: 0.7195 - focal_tversky_loss: 0.5500
Epoch 00001: val_loss improved from inf to 0.57999, saving model to saved_models/ISIC_2016/r2unet/usual_cutmix_mosaic_0/exp_1/model
90/90 [==============================] - 42s 431ms/step - loss: 0.5735 - dice_score: 0.4265 - iou: 0.2747 - dice_loss: 0.5735 - jaccard_loss: 0.7195 - focal_tversky_loss: 0.5500 - val_loss: 0.5800 - val_dice_score: 0.4200 - val_iou: 0.2715 - val_dice_loss: 0.5800 - val_jaccard_loss: 0.7247 - val_focal_tversky_loss: 0.5547 - lr: 0.0010
2022-02-28 20:43:14,793 - SKIN-CANCER-saved_models/ISIC_2016/r2unet/usual_cutmix_mosaic_0/exp_1 - INFO - Training is done model is saved in saved_models/ISIC_2016/r2unet/usual_cutmix_mosaic_0/exp_1
2022-02-28 20:43:16,063 - SKIN-CANCER-saved_models/ISIC_2016/r2unet/usual_cutmix_mosaic_0/exp_1 - INFO - Successfully saved figures to saved_models/ISIC_2016/r2un

# With Cutmix & Mosaic & hair aug

In [ ]:
train_identifier = "usual_cutmix_mosaic_hair_aug"

# Augmentation
hair_aug_p = 0.5
hair_rmv_p = 0

mosaic_p = 0.5
cutmix_p = 0.5
cutmix_beta = 1

In [ ]:
import shutil
import os
from utils.group_metrics import get_conf_mean_std

for model in model_names:
    model_path = os.path.join(save_path, dataset_name, model)
    for n in range(multi_train):
        seed = seeds + n
        saved_path_name = f"{train_identifier}_{n}"
        iter_model = os.path.join(model_path, saved_path_name)
        print(f"[INFO] Removing old model checkpoints: {iter_model}!!")
        shutil.rmtree(iter_model) if os.path.isdir(iter_model) else print(f'[INFO] {iter_model} does not exist!')
        !python train.py \
        --model $model --dataset_name $dataset_name \
        --save_path $save_path \
        --save_path_name $saved_path_name \
        --epochs $epochs \
        --lr $lr \
        --min_lr $min_lr \
        --reduce_lr_patience $reduce_lr_patience \
        --reduce_lr_factor $reduce_lr_factor \
        --early_stopping_p $early_stopping_p \
        --hair_aug_p $hair_aug_p \
        --hair_rmv_p $hair_rmv_p \
        --random_rotate_p $random_rotate_p \
        --p_horizontal_flip $p_horizontal_flip \
        --p_vertical_flip $p_vertical_flip \
        --p_center_crop $p_center_crop \
        --mosaic_p $mosaic_p \
        --cutmix_p $cutmix_p \
        --cutmix_beta $cutmix_beta \
        --seed $seed
    
        print(f"\n-----------------------------train {n} is done! for model: {model} ----------------------------------\n")
    try:
        csv_addresses = [
 os.path.join(save_path, dataset_name, model, f"{train_identifier}_{n}", "exp_1" , "csv_logger_train.csv") for n in range(multi_train)   
]
        metrics = get_mean_std(csv_addresses, 
                       arguments=(),
                       operators=(min, max, min, max, min, min, min, max, min, max, min, min),
                       save_path=os.path.join(save_path, dataset_name, model, "mean_std_metrics.csv")
                      )
        for name, metric in metrics.items():
            print(f"[INFO] Model: {model} == > {name}: {metric}")
    except Exception as e:
        print(f"[ERROR] {e} raised for {model_path}")

# With Cutmix & Mosaic & hair rmv

In [ ]:
train_identifier = "usual_cutmix_mosaic_hair_rmv"

# Augmentation
hair_aug_p = 0
hair_rmv_p = 0.5

mosaic_p = 0.5
cutmix_p = 0.5
cutmix_beta = 1

In [ ]:
import shutil
import os
from utils.group_metrics import get_conf_mean_std

for model in model_names:
    model_path = os.path.join(save_path, dataset_name, model)
    for n in range(multi_train):
        seed = seeds + n
        saved_path_name = f"{train_identifier}_{n}"
        iter_model = os.path.join(model_path, saved_path_name)
        print(f"[INFO] Removing old model checkpoints: {iter_model}!!")
        shutil.rmtree(iter_model) if os.path.isdir(iter_model) else print(f'[INFO] {iter_model} does not exist!')
        !python train.py \
        --model $model --dataset_name $dataset_name \
        --save_path $save_path \
        --save_path_name $saved_path_name \
        --epochs $epochs \
        --lr $lr \
        --min_lr $min_lr \
        --reduce_lr_patience $reduce_lr_patience \
        --reduce_lr_factor $reduce_lr_factor \
        --early_stopping_p $early_stopping_p \
        --hair_aug_p $hair_aug_p \
        --hair_rmv_p $hair_rmv_p \
        --random_rotate_p $random_rotate_p \
        --p_horizontal_flip $p_horizontal_flip \
        --p_vertical_flip $p_vertical_flip \
        --p_center_crop $p_center_crop \
        --mosaic_p $mosaic_p \
        --cutmix_p $cutmix_p \
        --cutmix_beta $cutmix_beta \
        --seed $seed
    
        print(f"\n-----------------------------train {n} is done! for model: {model} ----------------------------------\n")
    try:
        csv_addresses = [
 os.path.join(save_path, dataset_name, model, f"{train_identifier}_{n}", "exp_1" , "csv_logger_train.csv") for n in range(multi_train)   
]
        metrics = get_mean_std(csv_addresses, 
                       arguments=(),
                       operators=(min, max, min, max, min, min, min, max, min, max, min, min),
                       save_path=os.path.join(save_path, dataset_name, model, "mean_std_metrics.csv")
                      )
        for name, metric in metrics.items():
            print(f"[INFO] Model: {model} == > {name}: {metric}")
    except Exception as e:
        print(f"[ERROR] {e} raised for {model_path}")

_:)_